In [0]:
%sql

-- Widgets
CREATE WIDGET TEXT catalog DEFAULT "workspace";
CREATE WIDGET TEXT schema DEFAULT "dq_framework";

-- Catálogo  trabajo con valores dinámicos
EXECUTE IMMEDIATE 'USE CATALOG ' || getArgument('catalog');

-- Esquema de trabajo con valores dinámicos
EXECUTE IMMEDIATE 'CREATE SCHEMA IF NOT EXISTS ' || getArgument('schema');
EXECUTE IMMEDIATE 'USE SCHEMA '  || getArgument('schema');

-- --------------------------------------------------------------------
-- TABLA 1: dq_tables_config (Configuración de las tablas origen)
-- --------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS dq_tables_config (
  table_id STRING NOT NULL COMMENT 'Primary Key. Id de la tabla',
  table_name STRING NOT NULL COMMENT 'Nombre lógico de la tabla',
  table_name_tech STRING NOT NULL COMMENT 'Nombre técnico completo de la tabla de origen',
  primary_key STRING NOT NULL COMMENT 'Nombre de la(s) columna(s) que conforman la clave primaria de la tabla origen',
  staging_evidences_table STRING NOT NULL COMMENT 'Nombre de la tabla auxiliar para fallos en ejecución en paralelo'
)
USING DELTA
COMMENT 'Almacena los metadatos de las tablas sobre las que aplican las validaciones';


-- --------------------------------------------------------------------
-- TABLA 2: dq_rules_library (Catálogo de reglas)
-- --------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS dq_rules_library (
  rule_id STRING NOT NULL COMMENT 'PK. Código  de la regla',
  technical_rule_name STRING NOT NULL COMMENT 'Nombre técnico de la regla',
  functional_name STRING COMMENT 'Nombre funcional',
  description STRING COMMENT 'Descripción de lo que hace la regla',
  rule_level STRING COMMENT 'Define si es una regla para fila o para dataset',
  dimension_dq STRING COMMENT 'Dimensión de calidad',
  implementation_type STRING COMMENT 'Tipo de implementación (built-in, custom)',
  tags ARRAY<STRING> COMMENT 'Clasificación lógica de la regla',
  type_rule STRING COMMENT 'Técnica/Funcional',
  owner STRING COMMENT 'Propietario de la regla',
  updated_at TIMESTAMP COMMENT 'Última fecha en la que se actualizó la regla',
  updated_by STRING COMMENT 'Usuario que actualizó la regla'
)
USING DELTA
COMMENT 'Catálogo de reglas. Define qué es cada regla disponible';


-- --------------------------------------------------------------------
-- TABLA 3: dq_validations_catalog (Catálogo de validaciones)
-- --------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS dq_validations_catalog (
  validation_id STRING NOT NULL COMMENT 'PK. Código de la validación',
  rule_id STRING NOT NULL COMMENT 'FK. Código de la regla que se aplica',
  table_id STRING NOT NULL COMMENT 'FK. Código de la tabla sobre la que aplica',
  perimeter_definition STRING COMMENT 'Definición del ámbito sobre el que se aplica la validación',
  is_active BOOLEAN COMMENT 'Flag que indica si se ejecuta la validación o no',
  severity STRING COMMENT 'Nivel de impacto (informativo, warning, critico)',
  validation_definition STRING NOT NULL COMMENT 'Definición de la lógica y sus parámetros en formato JSON.',
  owner STRING COMMENT 'Propietario de la validación',
  tags ARRAY<STRING> COMMENT 'Clasificación lógica de la validación',
  updated_at TIMESTAMP COMMENT 'Fecha de actualización',
  updated_by STRING COMMENT 'Usuario que actualizó la validación',  
  domain STRING COMMENT 'Dominio de la tabla sobre el que aplica',
  Param_columns ARRAY<STRING> COMMENT '',
  Param_value STRING COMMENT '',
  Param_value_min STRING COMMENT '',
  Param_value_max STRING COMMENT '',
  Param_range STRING COMMENT '',
  Param_data_type STRING COMMENT '',
  Param_expression STRING COMMENT '',
  Param_query_SQL STRING COMMENT '',
  Param_merge_columns STRING COMMENT ''
)
USING DELTA
COMMENT 'Catálogo de validaciones. Cada fila es una aplicación de una regla a una columna de una tabla';

-- --------------------------------------------------------------------
-- TABLA 4: dq_execution_traceability (Trazabilidad de ejecuciones)
-- --------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS dq_execution_traceability (
  execution_id STRING NOT NULL COMMENT 'PK. Código de la ejecución',  
  table_id	STRING NOT NULL COMMENT 'FK. Código de la tabla sobre la que se está ejecutando',
  execution_timestamp TIMESTAMP COMMENT 'Fecha en la que se inicia la ejecución',
  execution_date DATE COMMENT 'Fecha de ejecución para particionamiento. Derivada de execution_timestamp',
  status STRING COMMENT 'Estado final: SUCCESS, FAILED, RUNNING.',
  duration_seconds DECIMAL(10, 2) COMMENT 'Tiempo en segundos que tarda la ejecución en realizarse',
  validations_executed INT COMMENT 'Total de validaciones ejecutadas en una ejecución',
  validations_failed INT COMMENT 'Total de validaciones con incidencias en una ejecución'
)
USING DELTA
PARTITIONED BY (execution_date)
COMMENT 'Registra cada ejecución del motor y su resumen global';

-- --------------------------------------------------------------------
-- TABLA 5: dq_validations_traceability (Trazabilidad de validaciones)
-- --------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS dq_validations_traceability (
  validation_trace_id STRING NOT NULL COMMENT 'PK. ID único de este resultado.',
  execution_id STRING NOT NULL COMMENT 'FK. Código de la ejecución',
  validation_id STRING NOT NULL COMMENT 'FK. Código de la validación ejecutada',
  rule_id STRING NOT NULL COMMENT 'FK. Código de la regla que se aplica',
  status STRING COMMENT 'Resultado de la regla: PASSED, FAILED, ERROR',
  perimeter BIGINT COMMENT 'Número de registros analizados',
  failed_records_count BIGINT COMMENT 'Conteo total de registros con incidencia',
  persistent_failures BIGINT COMMENT 'Incidencias que ya fallaron en la ejec. anterior',
  new_failures BIGINT COMMENT 'Incidencias que fallan por primera vez',
  resolved_failures BIGINT COMMENT 'Incidencias que fallaron antes pero se han corregido',
  execution_date DATE COMMENT 'Fecha de ejecución para particionamiento. Derivada de la ejecución'
)
USING DELTA
PARTITIONED BY (execution_date)
COMMENT 'Almacena el resultado de la ejecución para cada validación en cada ejecución';

-- --------------------------------------------------------------------
-- TABLA 6: dq_evidences (Evidencias de incidencias)
-- ------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS dq_evidences (
  evidence_id STRING NOT NULL COMMENT 'PK. ID único del registro fallido',
  execution_id STRING NOT NULL COMMENT 'FK. Código de la ejecución',
  validation_id STRING NOT NULL COMMENT 'FK. Código de la validación',
  failed_value STRING COMMENT 'Valor del campo que ha fallado.',
  table_pk STRING COMMENT 'Valor de la clave primaria del registro fallido',
  is_new_failure BOOLEAN COMMENT 'Indica si es la primera vez que este registro falla',
  execution_date DATE COMMENT 'Fecha de ejecución para particionamiento. Derivada de la ejecución'
)
USING DELTA
PARTITIONED BY (execution_date) -- <-- Particionamos por la nueva columna 'execution_date'
COMMENT 'Almacena cada registro individual que ha fallado de cada tabla';